In [ ]:
import os
import re
logdir = 'maya_logs'
filelist = os.listdir(logdir)
labels = {
    'blackscholes':0,
    'bodytrack':1,
    'canneal':2,
    'freqmine':3,
    'vips':4,
    'streamcluster':5,
    'splash2x.radiosity':6,
    'splash2x.volrend':7,
    'splash2x.water_nsquared':8,
    'splash2x.water_spatial':9
}
namereader = re.compile(r"(.+)_(\d+)_log\.txt")
m = namereader.match(filelist[5])
m.group(1)

In [ ]:
len(filelist)

In [ ]:
import random
import numpy as np
y_data = []
x_data = []
minlen = 3000
totpower = 0.0
cutoff = 450
mse = 0.0
for filename in filelist:
    sqe = 0.0
    m = namereader.match(filename)
    if not m:
        continue
    label = labels[m.group(1)]
    trace = []
    with open(logdir+'/'+filename,'r') as f:
        cnt = 0
        for line in f:
            if cnt == 0:
                cnt = 1
                continue
            tokens = line.split(" ")
            if(float(tokens[1]) < 0):
                continue
            trace.append(float(tokens[1]))
            sqe += (float(tokens[1]) - float(tokens[5]))**2
            cnt += 1
    tracelen = len(trace)
    mse += np.sqrt(sqe/tracelen)/len(filelist)
    if tracelen < cutoff:
        print(tracelen)
        print(filename)
        continue
    minlen = min(tracelen, minlen)
    offset = random.randint(0,tracelen-cutoff)
    offset = min(offset, random.randint(0,50))
    if np.array(trace).mean() > 0:
        totpower = totpower + np.array(trace).mean()
    else:
        print(filename)
    y_data.append(label)
    x_data.append(trace[offset:offset+cutoff])

print(minlen)
print(totpower/len(filelist))
print("MSE: {:.4f}".format(mse))

In [ ]:
split=1500
from sklearn import svm
clf = svm.SVC(gamma='auto',kernel='linear')
clf.fit(x_data[:-split], y_data[:-split])
pred_y = clf.predict(x_data[-split:])


In [ ]:
correct = [yp == y for yp,y in zip(pred_y, y_data[-split:])]
sum(correct)/len(correct)

In [ ]:
pred_y[:20]

In [ ]:
y_data[:20]